In [10]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import shutil
import torch

from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

In [11]:

def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

def load_modality_output(seed, projection_dim, batch_size, modality, task, suffix) : 
    result_dir = Path(f'Results/Linear_{modality}/Seed_{seed}/{task}_proj_{projection_dim}_batch_{batch_size}{suffix}')
    output = load_pickle(result_dir / 'outputs' / 'best_epoch.pkl')
    etf = Path(f'Results/ETF/ETF_{projection_dim}_IN_2_OUT_Seed_{seed}.pt')
    etf = torch.load(etf)
    return output, etf

def unimodal_process(output, key):
    data = output.get(key)
    features = data['features']
    return features

def bimodal_process(output1, output2, key, weight1=0.5, weight2=0.5) : 
    data1 = output1.get(key)
    data2 = output2.get(key)
    features1 = data1['features']
    features2 = data2['features']
    # combined_features = (features1 + features2) / 2
    combined_features = (features1 * weight1 + features2 * weight2) / (weight1 + weight2)
    return features1, features2, combined_features

def trimodal_process(output1, output2, output3, key, weight1=0.5, weight2=0.5, weight3=0.5) : 
    data1 = output1.get(key)
    data2 = output2.get(key)
    data3 = output3.get(key)
    features1 = data1['features']
    features2 = data2['features']
    features3 = data3['features']
    feature_12 = (features1 * weight1 + features2 * weight2) / (weight1 + weight2)
    feature_13 = (features1 * weight1 + features3 * weight3) / (weight1 + weight3)
    feature_23 = (features2 * weight2 + features3 * weight3) / (weight2 + weight3)
    combined_features = (features1 * weight1 + features2 * weight2 + features3 * weight3) / (weight1 + weight2 + weight3)
    return features1, features2, features3, feature_12, feature_13, feature_23, combined_features

def whole_process(output1, output2, output3, key) : 
    data1 = output1.get(key, None)
    data2 = output2.get(key, None)
    data3 = output3.get(key, None)
    # ignore none and agg the rest
    features_list = [data['features'] for data in [data1, data2, data3] if data is not None]
    features = np.mean(features_list, axis=0) if features_list else None
    return features

def get_output(modality, total_outputs, etf) : 
    labels, features = [], []
    labels = [values['labels'] for values in total_outputs.values() if values[modality] is not None]
    features = [[values[modality]] for values in total_outputs.values() if values[modality] is not None]
    labels = np.array(labels)
    features = np.concatenate(features, axis=0)
    probs = features @ etf.numpy()
    print(f"{modality} - Features shape: {features.shape}, Probs shape: {probs.shape}, Labels shape: {labels.shape}")
    return labels, probs

def get_performance(labels, probs):
    roc_auc = roc_auc_score(labels, probs)
    # pr_auc = average_precision_score(labels, probs)
    precision, recall, _ = precision_recall_curve(labels, probs)
    pr_auc = auc(recall, precision)
    return roc_auc, pr_auc, len(labels)

def get_score(modality, total_outputs, etf) : 
    labels, probs = get_output(modality, total_outputs, etf)
    roc_auc, pr_auc, n = get_performance(labels, probs[:,1])
    number_of_cases = (labels == 1).sum()
    return roc_auc, pr_auc, n, number_of_cases


In [15]:
projection_dim=128
batch_size=512
seed=2026
task='mortality_90days'
# task='readmission_15days'

suffix = '_FINETUNE'

total_scores = []
for seed in range(2026, 2029) : 

    tabular_outputs, proto = load_modality_output(seed, projection_dim, batch_size, 'tabular', task, suffix='')
    lab_outputs, _ = load_modality_output(seed, projection_dim, batch_size, 'lab', task, suffix='')
    note_outputs, _ = load_modality_output(seed, projection_dim, batch_size, 'note', task, suffix='_FINETUNE')
    split_mode = 'test'

    tab_output = tabular_outputs[split_mode]['outputs']
    lab_output = lab_outputs[split_mode]['outputs']
    note_output = note_outputs[split_mode]['outputs']

    tab_keys = set(tab_output.keys())
    lab_keys = set(lab_output.keys())
    note_keys = set(note_output.keys())

    # 3개의 조합이면, 나올 수 있는 경우의 수
    ## Tab > Lab & Tab > Note 포함관계
    # 1.tab_total
    tab_keys
    # 2.lab_total
    lab_keys
    # 3.note_total
    note_keys
    # 4. tab - lab - note
    tab_only_keys = tab_keys - lab_keys - note_keys
    # 5. lab - note
    lab_only_keys = lab_keys - note_keys
    # 6. note - lab
    note_only_keys = note_keys - lab_keys
    # 7. lab ∩ note
    lab_note_keys = lab_keys.intersection(note_keys)
    # 8. total
    total_keys = tab_keys.union(lab_keys).union(note_keys)

    print(f"1. tab total: {len(tab_keys)}")
    print(f"2. lab total: {len(lab_keys)}")
    print(f"3. note total: {len(note_keys)}")
    print(f"4. tab only: {len(tab_only_keys)}")
    print(f"5. lab only: {len(lab_only_keys)}")
    print(f"6. note only: {len(note_only_keys)}")
    print(f"7. lab & note: {len(lab_note_keys)}")
    print(f"8. total: {len(total_keys)}")

    total_outputs = {}
    for key in total_keys:
        ### Unimodal feature extraction
        # 1. tab total
        tab_features = unimodal_process(tab_output, key) if key in tab_keys else None
        # 2. lab total
        lab_features = unimodal_process(lab_output, key) if key in lab_keys else None
        # 3. note total
        note_features = unimodal_process(note_output, key) if key in note_keys else None    
        # 4. tab only
        tab_only_features = unimodal_process(tab_output, key) if key in tab_only_keys else None
        
        ### Bimodal feature extraction
        # 5. lab only : lab_only -> lab_only를 tab이 본 관점
        lab_only_lab_features, lab_only_tab_features, lab_only_features = bimodal_process(lab_output, tab_output, key, weight1=0.5, weight2=0.5) if key in lab_only_keys else (None, None, None)
        # lab_only_lab_features, lab_only_tab_features, lab_only_features = bimodal_process(lab_output, tab_output, key, weight1=lab_score, weight2=tab_score) if key in lab_only_keys else (None, None, None)
        
        # 6. note only : note_only -> note_only를 tab이 본 관점
        note_only_note_features, note_only_tab_features, note_only_features = bimodal_process(note_output, tab_output, key, weight1=0.5, weight2=0.5) if key in note_only_keys else (None, None, None)
        # note_only_note_features, note_only_tab_features, note_only_features = bimodal_process(note_output, tab_output, key, weight1=note_score, weight2=tab_score) if key in note_only_keys else (None, None, None)

        ### Trimodal feature extraction
        # 7. lab ∩ note
        mm_tab_features, mm_lab_features, mm_note_features, mm_tab_lab_features, mm_tab_note_features, mm_lab_note_features, mm_all_features = trimodal_process(tab_output, lab_output, note_output, key, weight1=0.5, weight2=0.5, weight3=0.5) if key in lab_note_keys else (None, None, None, None, None, None, None)
        # mm_tab_features, mm_lab_features, mm_note_features, mm_tab_lab_features, mm_tab_note_features, mm_lab_note_features, mm_all_features = trimodal_process(tab_output, lab_output, note_output, key, weight1=tab_score, weight2=lab_score, weight3=note_score) if key in lab_note_keys else (None, None, None, None, None, None, None)

        # 8. total
        total_features = whole_process(tab_output, lab_output, note_output, key)
        
        # Label
        label = tab_output.get(key)['labels']
        
        # Store results
        total_outputs[key] = {
            'tabular': tab_features,
            'lab': lab_features,
            'note': note_features,
            'tab_only': tab_only_features,
            'lab_only_lab': lab_only_lab_features, 'lab_only_tab': lab_only_tab_features, 'lab_only': lab_only_features,
            'note_only_note': note_only_note_features, 'note_only_tab': note_only_tab_features, 'note_only': note_only_features,
            'mm_tab': mm_tab_features, 'mm_lab': mm_lab_features, 'mm_note': mm_note_features,
            'mm_tab_lab': mm_tab_lab_features, 'mm_tab_note': mm_tab_note_features, 'mm_lab_note': mm_lab_note_features,
            'mm_all': mm_all_features,
            'total': total_features,
            'labels' : label,
        }
        

    # modality keys : 
    ## 1. unimodal : tabular, lab, note
    ## 2. bimodal : tab_only, lab_only, note_only, mm_tab, mm_lab, mm_note
    ## 3. trimodal : mm_tab_lab, mm_tab_note, mm_lab_note, mm_all
    ## 4. total : total


    # Unimodal
    total_scores.append([seed, 'Tabular', *get_score('tabular', total_outputs, proto)])
    total_scores.append([seed, 'Lab', *get_score('lab', total_outputs, proto)])
    total_scores.append([seed, 'Note', *get_score('note', total_outputs, proto)])
    # Bimodal
    total_scores.append([seed, 'Tab Only', *get_score('tab_only', total_outputs, proto)])
    total_scores.append([seed, 'Lab Only', *get_score('lab_only', total_outputs, proto)])
    total_scores.append([seed, 'Note Only', *get_score('note_only', total_outputs, proto)])
    total_scores.append([seed, 'MM Tab', *get_score('mm_tab', total_outputs, proto)])
    total_scores.append([seed, 'MM Lab', *get_score('mm_lab', total_outputs, proto)])
    total_scores.append([seed, 'MM Note', *get_score('mm_note', total_outputs, proto)])
    # Trimodal
    total_scores.append([seed, 'MM Tab & Lab', *get_score('mm_tab_lab', total_outputs, proto)])
    total_scores.append([seed, 'MM Tab & Note', *get_score('mm_tab_note', total_outputs, proto)])
    total_scores.append([seed, 'MM Lab & Note', *get_score('mm_lab_note', total_outputs, proto)])
    total_scores.append([seed, 'MM All', *get_score('mm_all', total_outputs, proto)])
    # Total
    total_scores.append([seed, 'Total', *get_score('total', total_outputs, proto)])

total_score_df = pd.DataFrame(total_scores, columns=['Seed', 'Modality', 'ROC-AUC', 'PR-AUC', 'N', 'Number of Cases'])
total_score_df_balanced = total_score_df.copy()

1. tab total: 32945
2. lab total: 27286
3. note total: 20273
4. tab only: 4574
5. lab only: 8098
6. note only: 1085
7. lab & note: 19188
8. total: 32945
tabular - Features shape: (32945, 128), Probs shape: (32945, 2), Labels shape: (32945,)
lab - Features shape: (27286, 128), Probs shape: (27286, 2), Labels shape: (27286,)
note - Features shape: (20273, 128), Probs shape: (20273, 2), Labels shape: (20273,)
tab_only - Features shape: (4574, 128), Probs shape: (4574, 2), Labels shape: (4574,)
lab_only - Features shape: (8098, 128), Probs shape: (8098, 2), Labels shape: (8098,)
note_only - Features shape: (1085, 128), Probs shape: (1085, 2), Labels shape: (1085,)
mm_tab - Features shape: (19188, 128), Probs shape: (19188, 2), Labels shape: (19188,)
mm_lab - Features shape: (19188, 128), Probs shape: (19188, 2), Labels shape: (19188,)
mm_note - Features shape: (19188, 128), Probs shape: (19188, 2), Labels shape: (19188,)
mm_tab_lab - Features shape: (19188, 128), Probs shape: (19188, 2), L

In [16]:
# avg
avg_scores = total_score_df_balanced.groupby('Modality').agg({'ROC-AUC': ['mean', 'std'], 'PR-AUC': ['mean', 'std'], 'N': 'mean', 'Number of Cases': 'mean'}).reset_index()
avg_scores.columns = ['Modality', 'ROC-AUC Mean', 'ROC-AUC Std', 'PR-AUC Mean', 'PR-AUC Std', 'N', 'Number of Cases']
avg_scores[['N', 'Number of Cases']] = avg_scores[['N', 'Number of Cases']].astype(int)
avg_scores['case_ratio'] = avg_scores['Number of Cases'] / avg_scores['N']
# avg_scores = avg_scores.sort_values(by='ROC-AUC Mean', ascending=False)
avg_scores[['Modality', 'Number of Cases', 'N', 'case_ratio', 'ROC-AUC Mean', 'PR-AUC Mean']]

target_modality_and_order = ['Tabular', 'Lab', 'Note', # Unimodal
                             'MM All', # Trimodal
                             'Tab Only', 'Lab Only', 'Note Only', # Bimodal - only
                             'Total', # Total
                             ]
avg_scores = avg_scores.set_index('Modality').loc[target_modality_and_order].reset_index()
avg_scores[['ROC-AUC Mean', 'PR-AUC Mean']] = avg_scores[['ROC-AUC Mean', 'PR-AUC Mean']].apply(lambda x : round(x * 100, 2))
avg_scores[['Modality', 'Number of Cases', 'N', 'case_ratio', 'ROC-AUC Mean', 'PR-AUC Mean']]
avg_scores[['Modality', 'Number of Cases', 'N', 'case_ratio', 'ROC-AUC Mean', 'PR-AUC Mean']].to_csv(f'Results/Whole_{task.capitalize()}.csv', index=False)
avg_scores

,Modality,ROC-AUC Mean,ROC-AUC Std,PR-AUC Mean,PR-AUC Std,N,Number of Cases,case_ratio
0,Tabular,81.77,0.002353,42.03,0.000294,32945,3050,0.092579
1,Lab,74.83,0.000949,30.22,0.004262,27268,2843,0.104261
2,Note,82.19,0.000840,47.71,0.003805,20280,2202,0.108580
3,MM All,84.71,0.005800,52.35,0.009976,19157,2143,0.111865
4,Tab Only,66.43,0.017455,19.70,0.063897,4554,148,0.032499
5,Lab Only,85.81,0.009401,46.40,0.025023,8111,700,0.086303
6,Note Only,90.15,0.018545,66.14,0.038855,1122,58,0.051693
7,Total,84.50,0.003259,49.63,0.003340,32945,3050,0.092579


In [18]:
# avg
avg_scores = total_score_df_balanced.groupby('Modality').agg({'ROC-AUC': ['mean', 'std'], 'PR-AUC': ['mean', 'std'], 'N': 'mean', 'Number of Cases': 'mean'}).reset_index()
avg_scores.columns = ['Modality', 'ROC-AUC Mean', 'ROC-AUC Std', 'PR-AUC Mean', 'PR-AUC Std', 'N', 'Number of Cases']
avg_scores[['N', 'Number of Cases']] = avg_scores[['N', 'Number of Cases']].astype(int)
avg_scores['case_ratio'] = avg_scores['Number of Cases'] / avg_scores['N']
# avg_scores = avg_scores.sort_values(by='ROC-AUC Mean', ascending=False)
avg_scores[['Modality', 'Number of Cases', 'N', 'case_ratio', 'ROC-AUC Mean', 'PR-AUC Mean']]

target_modality_and_order = ['MM Tab', 'MM Lab', 'MM Note', # Bimodal - cross
                             'MM Tab & Lab', 'MM Tab & Note', 'MM Lab & Note', # Trimodal - pair
                            #  'Tabular', 'Lab', 'Note', # Unimodal
                             'MM All', # Trimodal - all
                             ]
avg_scores = avg_scores.set_index('Modality').loc[target_modality_and_order].reset_index()
avg_scores[['ROC-AUC Mean', 'PR-AUC Mean']] = avg_scores[['ROC-AUC Mean', 'PR-AUC Mean']].apply(lambda x : round(x * 100, 2))
avg_scores[['Modality', 'Number of Cases', 'N', 'case_ratio', 'ROC-AUC Mean', 'PR-AUC Mean']]
avg_scores[['Modality', 'Number of Cases', 'N', 'case_ratio', 'ROC-AUC Mean', 'PR-AUC Mean']].to_csv(f'Results/MM_{task.capitalize()}.csv', index=False)
avg_scores

,Modality,ROC-AUC Mean,ROC-AUC Std,PR-AUC Mean,PR-AUC Std,N,Number of Cases,case_ratio
0,MM Tab,80.81,0.002657,42.74,0.010450,19157,2143,0.111865
1,MM Lab,73.25,0.003090,30.16,0.010745,19157,2143,0.111865
2,MM Note,81.67,0.000103,47.21,0.002850,19157,2143,0.111865
3,MM Tab & Lab,82.41,0.009968,46.07,0.016778,19157,2143,0.111865
4,MM Tab & Note,84.06,0.002679,50.21,0.007125,19157,2143,0.111865
5,MM Lab & Note,82.70,0.002984,49.16,0.003327,19157,2143,0.111865
6,MM All,84.71,0.005800,52.35,0.009976,19157,2143,0.111865
